In [1]:
import cv2
import numpy as np
from ultralytics import YOLO, SAM
from PIL import Image, ImageFilter
import pandas as pd

# Load models
model = YOLO('./runs/detect/train27/weights/best.pt')
sam_model = SAM('sam2.1_b.pt')

In [2]:
# Load image using PIL
image_pil = Image.open("./test/f1.jpg").convert("RGB")
image_np = np.array(image_pil)

# Create blurred image using PIL
blurred_image_pil = image_pil.filter(ImageFilter.GaussianBlur(radius=25)) 
blurred_image_np = np.array(blurred_image_pil)


# Object detection and processing
results = model(image_np)

for result in results:
    boxes = result.boxes
    for box in boxes:
        cls = int(box.cls)
        conf = box.conf
        if model.names[cls] == "Face" and conf > 0.5:
            xyxy = box.xyxy[0].tolist()
            x1, y1, x2, y2 = map(int, xyxy)

            # SAM segmentation
            input_box = np.array([x1, y1, x2, y2]).reshape(1, 4)
            sam_result = sam_model.predict(image_pil, bboxes=input_box)
            mask = sam_result[0].masks.data[0].cpu().numpy().astype(np.uint8)

            # Apply blur
            image_np[mask == 1] = blurred_image_np[mask == 1]

# Display image using PIL
image_pil_blurred = Image.fromarray(image_np)
image_pil_blurred.show()  # image_pil_blurred.save("output.jpg")


0: 640x640 6 Faces, 14.3ms
Speed: 2.2ms preprocess, 14.3ms inference, 70.7ms postprocess per image at shape (1, 3, 640, 640)

0: 1024x1024 1 0, 183.8ms
Speed: 2.9ms preprocess, 183.8ms inference, 8.8ms postprocess per image at shape (1, 3, 1024, 1024)

0: 1024x1024 1 0, 145.8ms
Speed: 2.7ms preprocess, 145.8ms inference, 0.4ms postprocess per image at shape (1, 3, 1024, 1024)

0: 1024x1024 1 0, 145.0ms
Speed: 2.7ms preprocess, 145.0ms inference, 0.4ms postprocess per image at shape (1, 3, 1024, 1024)

0: 1024x1024 1 0, 140.8ms
Speed: 2.8ms preprocess, 140.8ms inference, 0.4ms postprocess per image at shape (1, 3, 1024, 1024)

0: 1024x1024 1 0, 146.8ms
Speed: 2.7ms preprocess, 146.8ms inference, 0.4ms postprocess per image at shape (1, 3, 1024, 1024)

0: 1024x1024 1 0, 146.2ms
Speed: 2.6ms preprocess, 146.2ms inference, 0.4ms postprocess per image at shape (1, 3, 1024, 1024)


In [4]:
model = YOLO('/mnt/WD/ds-phase-2/09-cv/Cancer_3/last_axial_coronal/weights/best.pt')
sam_model = SAM('sam2.1_b.pt')


(process:49596): Gtk-WARNING **: 13:50:41.374: Locale not supported by C library.
	Using the fallback 'C' locale.
** Message: 13:50:41.405: main.vala:507: Starting session with system profile


In [10]:
image_pil = Image.open("/mnt/WD/09_data/c2/images/test/00072_81.jpg").convert("RGB")
image_np = np.array(image_pil)
# Create a transparent overlay (PIL Image)
overlay = Image.new("RGBA", image_pil.size, (0, 0, 0, 0))

# Object detection and processing
results = model(image_np)

for result in results:
    boxes = result.boxes
    for box in boxes:
        cls = int(box.cls)
        conf = box.conf
        if model.names[cls] == "negative" and conf > 0.4:
            xyxy = box.xyxy[0].tolist()
            x1, y1, x2, y2 = map(int, xyxy)

            # SAM segmentation
            input_box = np.array([x1, y1, x2, y2]).reshape(1, 4)
            sam_result = sam_model.predict(image_pil, bboxes=input_box)
            mask = sam_result[0].masks.data[0].cpu().numpy().astype(np.uint8)

            # Apply transparent overlay using PIL
            mask_pil = Image.fromarray(mask * 255).convert("L")
            overlay.paste((0, 255, 0, 128), box=None, mask=mask_pil)
            
        if model.names[cls] == "positive" and conf > 0.4:
            xyxy = box.xyxy[0].tolist()
            x1, y1, x2, y2 = map(int, xyxy)

            # SAM segmentation
            input_box = np.array([x1, y1, x2, y2]).reshape(1, 4)
            sam_result = sam_model.predict(image_pil, bboxes=input_box)
            mask = sam_result[0].masks.data[0].cpu().numpy().astype(np.uint8)

            # Apply transparent overlay using PIL
            mask_pil = Image.fromarray(mask * 255).convert("L")  # Convert mask to grayscale PIL image
            overlay.paste((255, 0, 0, 128), box=None, mask=mask_pil)  # Paste semi-transparent black over the mask


# Composite the original image and the overlay
image_with_overlay = Image.composite(overlay, image_pil, overlay)

# Display or save
image_with_overlay.show()
#image_with_overlay.save("output_with_overlay.png")



0: 512x512 1 negative, 3.4ms
Speed: 1.7ms preprocess, 3.4ms inference, 0.6ms postprocess per image at shape (1, 3, 512, 512)

0: 1024x1024 1 0, 151.9ms
Speed: 3.1ms preprocess, 151.9ms inference, 0.3ms postprocess per image at shape (1, 3, 1024, 1024)



(process:50690): Gtk-WARNING **: 13:53:00.479: Locale not supported by C library.
	Using the fallback 'C' locale.
** Message: 13:53:00.507: main.vala:507: Starting session with system profile
